In [9]:
import tensorflow as tf
import numpy as np

#### 1-1. cell 생성 및 rnn 구동 (input = (1,1,4))

In [26]:
x_data = np.array([[[1,0,0,0]]], dtype=np.float32)

#ht에 대한 dimension 정의
hidden_size = 2
print("입력 데이터 차원: ", x_data.shape)


with tf.variable_scope('lstm1', reuse=True):
    
    #1. cell 정의(ht에 대한 dimension 정의)
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
    
    #2. cell 구동
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

    print("출력 값: ", outputs.eval())
    print("출력 차원: ", outputs.eval().shape)

입력 데이터 차원:  (1, 1, 4)
출력 값:  [[[-0.08205257 -0.01191202]]]
출력 차원:  (1, 1, 2)


#### 1-2. cell 생성 및 rnn 구동(input = (1,5,4))

In [27]:
#ht에 대한 dimension 정의
hidden_size = 2

h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

x_data = np.array([[h,e,l,l,o]], dtype=np.float32)
print("입력 데이터 차원: ", x_data.shape)

with tf.variable_scope('lstm1', reuse=True):
    #1. cell 정의(ht에 대한 dimension 정의)
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
    
    #2. cell 구동
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

    print("출력 값: ", outputs.eval())
    print("출력 차원: ", outputs.eval().shape)

입력 데이터 차원:  (1, 5, 4)
출력 값:  [[[ 0.04653593 -0.09376747]
  [ 0.07885884 -0.1869417 ]
  [ 0.12290448 -0.07888173]
  [ 0.15421937  0.00226069]
  [ 0.13376617  0.1231476 ]]]
출력 차원:  (1, 5, 2)


#### 1-3. cell 생성 및 rnn 구동(input = (3,5,4))

In [28]:
#ht에 대한 dimension 정의
hidden_size = 2

h = [1,0,0,0]
e = [0,1,0,0]
l = [0,0,1,0]
o = [0,0,0,1]

x_data = np.array([[h,e,l,l,o],
                   [e,o,l,l,o],
                   [l,e,e,l,l]], dtype=np.float32)

print("입력 데이터 차원: ", x_data.shape)

with tf.variable_scope('lstm1', reuse=True):
    #1. cell 정의(ht에 대한 dimension 정의)
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size)
    
    #2. cell 구동
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype=tf.float32)

    sess = tf.InteractiveSession()
    sess.run(tf.global_variables_initializer())

    print("출력 값: ", outputs.eval())
    print("출력 차원: ", outputs.eval().shape)

입력 데이터 차원:  (3, 5, 4)
출력 값:  [[[ 0.08319841 -0.07219824]
  [-0.03352327 -0.10916224]
  [ 0.06199432 -0.04653881]
  [ 0.13939244 -0.0192358 ]
  [ 0.10877636 -0.04709471]]

 [[-0.09506787 -0.057366  ]
  [-0.05326831 -0.03307088]
  [ 0.05138043  0.00655901]
  [ 0.12849385  0.01301786]
  [ 0.10040542 -0.0195493 ]]

 [[ 0.09069875  0.01526689]
  [-0.03492649 -0.05159034]
  [-0.12483385 -0.09275628]
  [-0.03942011 -0.02740682]
  [ 0.06261769  0.00357484]]]
출력 차원:  (3, 5, 2)


#### 1-4. RNN 학습

In [51]:
tf.reset_default_graph()

In [52]:
sequence_length = 6
input_dim = 5
batch_size = 1

#1. 데이터 정의
idx2char = ['h', 'i', 'e', 'l', 'o']
x_data = [[0, 1, 0, 2, 3, 3]] #hihell
x_one_hot = [[[1,0,0,0,0],
              [0,1,0,0,0],
              [1,0,0,0,0],
              [0,0,1,0,0],
              [0,0,0,1,0],
              [0,0,0,1,0]]]

x_data = np.array(x_one_hot) 
y_data = np.array([[1,0,2,3,3,4]]) # ihello

print("입력 데이터 차원: ", x_data.shape)
print("목표 데이터 차원: ", y_data.shape)

X = tf.placeholder(tf.float32, [None, sequence_length, input_dim])
Y = tf.placeholder(tf.int32, [None, sequence_length])

#2. cell 정의
with tf.variable_scope('lstm1'):
    cell = tf.contrib.rnn.BasicRNNCell(num_units=input_dim)
    iniital_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=iniital_state, dtype=tf.float32)

    #3. cost function 정의
    weights = tf.ones([batch_size, sequence_length])
    sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
    loss = tf.reduce_mean(sequence_loss)

    #4. optimizer 정의
    train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

    #5. prediction 정의
    prediction = tf.argmax(outputs, axis=2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(2000):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        
        if i % 100 == 0:
            print(i, "loss: ", l, "prediction: ", result, "true Y: ", y_data)

            #print char using dic
            print(np.squeeze(result))
            result_str = [idx2char[c] for c in np.squeeze(result)]
            print('\tPrediction str: ', ''.join(result_str))

입력 데이터 차원:  (1, 6, 5)
목표 데이터 차원:  (1, 6)
0 loss:  1.69838 prediction:  [[4 0 4 2 3 3]] true Y:  [[1 0 2 3 3 4]]
[4 0 4 2 3 3]
	Prediction str:  ohoell
100 loss:  0.460376 prediction:  [[1 0 2 3 3 4]] true Y:  [[1 0 2 3 3 4]]
[1 0 2 3 3 4]
	Prediction str:  ihello
200 loss:  0.450198 prediction:  [[1 0 2 3 3 4]] true Y:  [[1 0 2 3 3 4]]
[1 0 2 3 3 4]
	Prediction str:  ihello
300 loss:  0.446881 prediction:  [[1 0 2 3 3 4]] true Y:  [[1 0 2 3 3 4]]
[1 0 2 3 3 4]
	Prediction str:  ihello
400 loss:  0.444936 prediction:  [[1 0 2 3 3 4]] true Y:  [[1 0 2 3 3 4]]
[1 0 2 3 3 4]
	Prediction str:  ihello
500 loss:  0.443614 prediction:  [[1 0 2 3 3 4]] true Y:  [[1 0 2 3 3 4]]
[1 0 2 3 3 4]
	Prediction str:  ihello
600 loss:  0.442638 prediction:  [[1 0 2 3 3 4]] true Y:  [[1 0 2 3 3 4]]
[1 0 2 3 3 4]
	Prediction str:  ihello
700 loss:  0.441877 prediction:  [[1 0 2 3 3 4]] true Y:  [[1 0 2 3 3 4]]
[1 0 2 3 3 4]
	Prediction str:  ihello
800 loss:  0.441261 prediction:  [[1 0 2 3 3 4]] true Y:  